# New Apps For Book Lovers?
### by Ferdinand Paul

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 

You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

### Task

- Find the number of books released after January 1, 2000.
- Find the number of user reviews and the average rating for each book.
- Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).
- Identify the author with the highest average book rating (look only at books with at least 50 ratings).
- Find the average number of text reviews among users who rated more than 50 books.


In [56]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [91]:
# create function for querys
def query(code):
    return pd.io.sql.read_sql(code, con = engine)

### Check Tables

In [187]:
# check BOOKS table
books_table = "SELECT * FROM BOOKS LIMIT 5" 
query(books_table)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [188]:
# check AUTHORS table
books_table = "SELECT * FROM AUTHORS LIMIT 5" 
query(books_table)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [189]:
# check RATINGS table
books_table = "SELECT * FROM RATINGS LIMIT 5" 
query(books_table)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [190]:
# check REVIEWS table
books_table = "SELECT * FROM REVIEWS LIMIT 5" 
query(books_table)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [191]:
# check PUBLISHERS table
books_table = "SELECT * FROM PUBLISHERS LIMIT 5" 
query(books_table)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Find the number of books released after January 1, 2000

In [192]:
books ='''
SELECT count(book_id) books 
FROM BOOKS WHERE publication_date > '2000-01-01'
'''
query(books)

,books
0,819


### Find the number of user reviews and the average rating for each book

In [193]:
ratings='''
SELECT count(re.review_id) as reviews, avg(ra.rating) as average_ratings 
FROM BOOKS as bo
JOIN reviews as re
ON re.book_id = bo.book_id
JOIN ratings as ra
ON bo.book_id = ra.book_id
'''
query(ratings)

,reviews,average_ratings
0,26167,3.936791


### Identify the publisher that has released the greatest number of books with more than 50 pages 

In [194]:
publisher ='''
SELECT pu.publisher, sum(bo.book_id) as books, bo.num_pages as pages 
FROM PUBLISHERS as pu
JOIN BOOKS as bo
ON bo.publisher_id = pu.publisher_id 
WHERE bo.num_pages > 50 
GROUP BY pu.publisher, bo.num_pages 
ORDER BY sum(bo.book_id) DESC
LIMIT 1
'''
query(publisher)

,publisher,books,pages
0,Penguin Books,2096,320


### Identify the author with the highest average book rating (look only at books with at least 50 ratings)

In [158]:
author = '''
SELECT au.author, avg(ra.rating) as avg_rating, count(ra.rating) as sum_ratings
FROM BOOKS as bo
JOIN authors as au
on au.author_id = bo.author_id
JOIN ratings as ra
ON bo.book_id = ra.book_id
GROUP BY au.author
HAVING count(ra.rating) >= 50
LIMIT 1
'''
query(author)

,author,avg_rating,sum_ratings
0,J.K. Rowling/Mary GrandPré,4.288462,312


### Find the average number of text reviews among users who rated more than 50 books

In [195]:
reviews = '''
SELECT COUNT(text)/COUNT(DISTINCT username) as avg_text_review
FROM reviews
WHERE username IN
(
SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(*) > 50
)
'''
query(reviews)

,avg_text_review
0,24
